In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset


dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset['train'].select(range(3000))





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(train_data[0]['article'])
print(train_data[0]['highlights'])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')


def cleantext(text):

  text=text.lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text)#removing special characters and digits
  tokens=word_tokenize(text)
  return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

nltk.download('punkt_tab')
train_dataset=train_data.to_pandas()
X_train=train_dataset['article']
y_train=train_dataset['highlights']
tokenized_docs=[cleantext(doc) for doc in X_train]
tokenized_summaries=[cleantext(s) for s in y_train]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
print(tokenized_docs[:2])
print(tokenized_summaries[:2])

[['london', 'england', 'reuters', 'harry', 'potter', 'star', 'daniel', 'radcliffe', 'gains', 'access', 'to', 'a', 'reported', 'million', 'million', 'fortune', 'as', 'he', 'turns', 'on', 'monday', 'but', 'he', 'insists', 'the', 'money', 'wont', 'cast', 'a', 'spell', 'on', 'him', 'daniel', 'radcliffe', 'as', 'harry', 'potter', 'in', 'harry', 'potter', 'and', 'the', 'order', 'of', 'the', 'phoenix', 'to', 'the', 'disappointment', 'of', 'gossip', 'columnists', 'around', 'the', 'world', 'the', 'young', 'actor', 'says', 'he', 'has', 'no', 'plans', 'to', 'fritter', 'his', 'cash', 'away', 'on', 'fast', 'cars', 'drink', 'and', 'celebrity', 'parties', 'i', 'dont', 'plan', 'to', 'be', 'one', 'of', 'those', 'people', 'who', 'as', 'soon', 'as', 'they', 'turn', 'suddenly', 'buy', 'themselves', 'a', 'massive', 'sports', 'car', 'collection', 'or', 'something', 'similar', 'he', 'told', 'an', 'australian', 'interviewer', 'earlier', 'this', 'month', 'i', 'dont', 'think', 'ill', 'be', 'particularly', 'extr

In [ ]:
# Download the GloVe vectors (you can choose other versions as well)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-05-12 14:55:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-12 14:55:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-12 14:55:53--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

In [ ]:
!pip install numpy
import numpy as np

# Load GloVe embeddings
def load_glove_embeddings(glove_file_path):
    embeddings = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Convert a tokenized document to GloVe vectors
def convert_to_vectors(tokens, glove_embeddings):
    vectors = []
    for token in tokens:
        if token in glove_embeddings:
            vectors.append(glove_embeddings[token])
        else:
            vectors.append(np.zeros(100))  # Use zero vector for unknown words
    return vectors


# Load GloVe 100d vectors (replace path with your downloaded file)
glove_file_path = 'glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings(glove_file_path)

# Convert tokenized documents into GloVe vectors
vectorized_docs = [convert_to_vectors(doc, glove_embeddings) for doc in tokenized_docs]
vectorized_summaries = [convert_to_vectors(s, glove_embeddings) for s in tokenized_summaries]

# Display the result
print(np.array(vectorized_docs[0]))  # Check the vectors for the first tokenized document


[[ 0.60553002 -0.050886   -0.15460999 ...  0.41466999  0.34641001
   0.019886  ]
 [-0.092902    0.43667999  0.63845998 ...  0.68544     0.28566
  -0.40700001]
 [-0.37518001 -0.75599998 -0.15465    ... -0.053903    0.37886
   0.35159999]
 ...
 [-0.22145     0.20658     0.51225001 ... -0.043929   -0.67294002
   0.11487   ]
 [ 0.31039     0.64859003  0.28481001 ... -0.88554001  0.91767001
  -0.57252997]
 [-0.56418997 -0.69006997  0.64247    ...  0.61062002 -0.20618001
  -0.035852  ]]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:

padded_docs = pad_sequences(vectorized_docs, maxlen=400, padding='post', truncating='post')
padded_summaries = pad_sequences(vectorized_summaries, maxlen=100, padding='post', truncating='post')

print(padded_docs.shape, padded_summaries.shape)  # Check the shape of padded sequences


(3000, 400, 100) (3000, 100, 100)


In [ ]:
from collections import Counter

# Combine tokenized documents and summaries to create a vocabulary
all_tokens = [word for doc in tokenized_docs + tokenized_summaries for word in doc]
word_counts = Counter(all_tokens)

max_vocab_size = 5000  # Limiting vocab size to 10,000 most frequent words
word2idx = {word: idx for idx, (word, _) in enumerate(word_counts.most_common(max_vocab_size))}


# Add a padding token
word2idx['<PAD>'] = 0

# Display the word2idx mapping (optional)
print(f"Word to index mapping: {list(word2idx.items())[:10]}")



Word to index mapping: [('the', 0), ('to', 1), ('of', 2), ('a', 3), ('and', 4), ('in', 5), ('said', 6), ('that', 7), ('for', 8), ('is', 9)]


In [ ]:
# Initialize the embedding matrix with zeros
embedding_dim = 100
embedding_matrix = np.zeros((len(word2idx), embedding_dim))

# Fill the embedding matrix with GloVe vectors
for word, idx in word2idx.items():
    if word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]
    else:
        embedding_matrix[idx] = np.zeros(embedding_dim)


print(embedding_matrix[:5])


[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.optimizers import Adam

# dimensions
embedding_dim = 100
latent_dim = 256
vocab_size = 5000
max_input_len = 400
max_summary_len =100

#encoder part
encoder_inputs = Input(shape=(max_input_len,))
enc_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                    input_length=max_input_len)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# decoder part
decoder_inputs = Input(shape=(max_summary_len,))
dec_emb = Embedding(input_dim=vocab_size+1 , output_dim=embedding_dim,
                    weights=[embedding_matrix], input_length=max_summary_len,
                    trainable=False)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#model details
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 400)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 400, 100)  │    500,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 100, 100)  │    500,100 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │    365,568 │ embedding_2[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 100,      │    365,568 │ embedding_3[0][0… │
│                     │ 256), (None,      │            │ lstm_2[0][1],     │
│                     │ 256), (None,      │            │ lstm_2[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 100, 5000) │  1,285,000 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,016,236 (11.51 MB)

 Trainable params: 2,516,136 (9.60 MB)

 Non-trainable params: 500,100 (1.91 MB)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

# Prepare the input data for the encoder and decoder
encoder_input_data = pad_sequences([[word2idx.get(w, 0) for w in doc] for doc in tokenized_docs],
                                   maxlen=max_input_len, padding='post')

decoder_input_data = pad_sequences([[word2idx.get(w, 0) for w in sumry[:-1]] for sumry in tokenized_summaries],
                                   maxlen=max_summary_len, padding='post')

decoder_target_data = pad_sequences([[word2idx.get(w, 0) for w in sumry[1:]] for sumry in tokenized_summaries],
                                    maxlen=max_summary_len, padding='post')


# Train the model
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=8,
          epochs=20,
          validation_split=0.2)
#model was already runned for 20 epochs and i have stopped the training and saved the model see below

Epoch 1/20
  3/300 ━━━━━━━━━━━━━━━━━━━━ 18:50 4s/step - accuracy: 0.7180 - loss: 1.5743

KeyboardInterrupt: 

In [ ]:
model.save('summarization_model.h5')


In [ ]:
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


test_data = dataset['test'].select(range(500))
test_dataset=test_data.to_pandas()
test_articles=test_dataset['article']
test_summaries=test_dataset['highlights']

# 1. Tokenize test articles and summaries
X_test_tokens = [word_tokenize(doc.lower()) for doc in test_articles]
y_test_tokens = [word_tokenize(sumry.lower()) for sumry in test_summaries]

# 2. Convert tokens to sequences using your existing word_index


def tokens_to_indices(token_list, word2idx, oov_token='unk'):
    return [word2idx.get(token, word2idx.get(oov_token, 1)) for token in token_list]

X_test_sequences = [tokens_to_indices(tokens, word2idx) for tokens in X_test_tokens]
y_test_sequences = [tokens_to_indices(tokens, word2idx) for tokens in y_test_tokens]


# 3. Pad sequences (use same maxlen as training)
X_test_padded = pad_sequences(X_test_sequences, maxlen=400, padding='post', truncating='post')
y_test_padded = pad_sequences(y_test_sequences, maxlen=101, padding='post', truncating='post')

# 4. Load the model
from tensorflow.keras.models import load_model
model = load_model('summarization_model.h5')


# Now this works
decoder_input_test = y_test_padded[:, :-1]  # (None, 100)
decoder_target_test = y_test_padded[:, 1:]  # (None, 100)

loss, accuracy = model.evaluate([X_test_padded, decoder_input_test], decoder_target_test, verbose=1)

print(f"Test Accuracy: {accuracy:.4f}, Test Loss: {loss:.4f}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.6550 - loss: 2.3211
Test Accuracy: 0.6557, Test Loss: 2.3196


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=47bd1497ce07fa26c1745e46bd63159092c2a6003342296961d7647ca5171d6f
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
